# 第14章 编写数值计算程序

## 14.1 问题

问题描述：在一个计算旅行商漫游点集的程序，要求计算K维空间中点与点之间的欧氏距离。例如，三维空间中的点$(a_1,a_2,a_3)$和$(b_1, b_2, b_3)$之间的距离是：
$$
\sqrt{(a_1 - b_1)^2 + (a_2 - b_2)^2 + (a_3 - b_3)^2}
$$

**程序1：**

In [1]:
import math

def calc_K_dist1(A, B):
    Sum = 0
    for a, b in zip(A, B):
        t = a - b
        Sum += t * t
    return math.sqrt(Sum)

In [2]:
A = [1, 2, 3]
B = [4, 5, 6]
calc_K_dist1(A, B)

5.196152422706632

缺点：
- 可能产生溢出：所有的输入、中间计算的差以及输出都需要在有效范围内。
- 开销比较大：耗时最长的部分是计算平方根。

目标：寻找一个K维欧氏距离的计算函数，需要具有以下性质：
- 定义域：K在1\~16范围之内，点的坐标是单精度的。
- 精度：单精度的输出应该精确到十进制的最后一位。
- 健壮性：避免上溢出和下溢出。
- 性能：程序应该尽可能地快。

## 14.2 牛顿迭代

该方法在《计算机的构造与解释》一书中也有详细的介绍，求$f(x) = x^2 - a$的零点，即求平方根。

该方法并不显式地计算区间，而是从一个猜测的$x_0$开始，生成一个逼近序列$x_1,x_2,x_3,\cdots$，为了生成$x_{i+1}$需要直到$f(x_i)$和其导数$f'(x_i)$，迭代式如下：
$$
x_{i+1} = x_i - f(x_i) / f'(x_i)
$$

根据上述公式，为了计算$\sqrt{a}$，求解$f(x) = x^2 - a$的零点，已知$f'(x) = 2x$，则牛顿迭代的公式为：
$$
\begin{aligned}
x_{i + 1} 
& = x_i - (x_i^2 - a) / 2x_i \\
&= x_i - x_i /2 + a/ 2x_i \\
&= (x_i + a / x_i) / 2
\end{aligned}
$$

可知，如果$x_i = \sqrt{a}$，则
$$
x_{i + 1} = (\sqrt{a} + a / \sqrt{a}) / 2 = \sqrt{a}
$$

## 14.3 良好的起点

关于牛顿迭代，有以下两个问题：
1. $x_0$的初始值选择多少合适？
2. 把$x_i$作为最后结果之前，应该进行多少次迭代？

牛顿迭代中的每一次迭代都会使精确位数翻倍，因为第$i+1$步的误差是与第$i$步的误差的平方成比例的，满足平方收敛，具备条件：
1. 导数不接近零。
2. 初始的猜测值必须足够接近最后的根。

## 14.4 代码

**程序2：**

In [7]:
def calc_K_dist2(A, B):
    t = abs(A[0] - B[0])
    max_ = t
    sum_ = t*t
    for j in range(1, len(A)):
        t = abs(A[j] - B[j])
        if t > max_:
            max_ = t
        sum_ += t * t
    
    eps = 1e-7
    z = max_
    while True:
        new_z = 0.5 * (z + sum_ / z)
        if abs(new_z - z) < eps * new_z:
            break
        z = new_z
    
    return new_z

In [8]:
A = [1, 2, 3]
B = [4, 5, 6]
calc_K_dist2(A, B)

5.196152422706632

**程序3：**

In [10]:
def calc_K_dist3(A, B):
    t = abs(A[0] - B[0])
    max_ = t
    sum_ = t*t
    for j in range(1, len(A)):
        t = abs(A[j] - B[j])
        if t > max_:
            max_ = t
        sum_ += t * t
    
    max_ = max_ * 2
    max_ = 0.5 * (max_ + sum_ / max_)
    max_ = 0.5 * (max_ + sum_ / max_)
    max_ = 0.5 * (max_ + sum_ / max_)
    return 0.5 * (max_ + sum_ / max_)

In [11]:
A = [1, 2, 3]
B = [4, 5, 6]
calc_K_dist3(A, B)

5.196152422706632

## 14.5 原理

- 上下文环境的重要性：得到一个快速的距离程序的过程受到许多因素的影响。
- 牛顿迭代
- 编写代码的技巧：尽管大的改进通常归功于算法的改变，但是代码的小改进也能减少运行时间。
- 库函数的作用：为了换取速度，需要牺牲可复用性和数值精确度，在工程上还算明智。